In [1]:
from multiprocessing import Process

In [2]:
def funcao(name):
    print("hello, sou ", name)

In [3]:
p = Process(target=funcao, args=("bob filho", )) # tem que ter essa virgula e vazio no fim pra dizer que acabou. 
# dá um erro bizarro se não fizer
p.start()
print("hello, sou ", "bob pai")
funcao("eu")
p.join()

hello, sou  bob filho
hello, sou  bob pai
hello, sou  eu


In [4]:
# Fazer em 4x paralelo
def funcao2(name, i):
    print("hello, sou ", name, i)

procs = []
for i in range(4):
    p = Process(target=funcao2, args=("bob filho", i,  ))
    procs.append(p)

print("hello, sou ", "bob pai")

for i in range(4):
    procs[i].start()
for i in range(4):
    procs[i].join() # o join tem que estar em for separado pra ele poder juntar tudo depois de executar.
    # se não executaria um por vez, não sendo paralelo

# dá um negócio louco pq tá executando os prints fora de ordem

hello, sou  bob pai
hello, sou  bob filhohello, sou  0hello, sou  bob filho 
 2bob filho hello, sou 
 bob filho 1
3


In [5]:
#calculo de pi sequencial
import time
def pi_naive(start, end, step): # calcula pi somando área de retângulos
    print("Start: ", str(start))
    print("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    return soma

num_steps = 100000000
sums = 0.0
step = 1.0/num_steps
tic = time.time() #tempo inicial
sums = pi_naive(0, num_steps, step)
toc = time.time() #tempo final
pi = step*sums
print("Valor pi: %.10f" %pi)
print("Tempo pi: %.8f s" %(toc-tic)) # calcula tempo de execução

Start:  0
End:  100000000
Valor pi: 3.1415926536
Tempo pi: 8.95420790 s


In [6]:
#calculo de pi paralelo (feito pelo prof)
from multiprocessing import Process
import time
PROCS = 2
def pi(start, end, step):
    print ("Start: ", str(start))
    print ("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    print(soma)

num_steps = 100000000 #100.000.000
soma = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
workers = []

for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, ))
    workers.append(worker)
for worker in workers :
    worker.start()
for worker in workers :
    worker.join()
toc = time.time()
print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  Start: 0 
End:  49999999
50000000
End:  99999999
128700219.75867517
185459040.40031576
Tempo Pi: 4.68262029 s


In [7]:
#calculo de pi paralelo (diferente do prof) isso aqui é 10x mais difícil que usar pipes
#, que é a solução do professor logo abaixo
import multiprocessing


def pi_naive(start, end, step, proc_num, return_list): # calcula pi somando área de retângulos
    print("Start: ", str(start))
    print("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    return_list.append(soma)


if __name__ == "__main__":
    start = 0
    end = 100000000 # 100.000.000
    sums = 0.0
    step = 1.0/end
    manager = multiprocessing.Manager()
    return_list = manager.list()
    jobs = []
    tic = time.time()
    for proc_num, start, end in zip([0, 1], [0, end//2], [end//2, end]):
        p = multiprocessing.Process(target=pi_naive, args=(start, end, step  ,proc_num, return_list))
        jobs.append(p)
        p.start()

    for proc in jobs:
        proc.join()
    toc = time.time()
    print ("Tempo Pi: %.8f s" %(toc-tic))
    print(return_list[0] + return_list[1])

Start:  0
Start: End:  50000000 50000000
End:  100000000

Tempo Pi: 4.67741156 s
314159265.35899097


In [35]:
# paralelo usando pipe
from multiprocessing import Process, Pipe
import sys
import time
PROCS = 8
def pi(start, end, step, entrada):
    print ("Start: ", str(start))
    print ("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    entrada.send(soma) # manda os resultados pro pipe assim que pronto
    print(soma)
    
entrada , saida = Pipe()
num_steps = 100000000 #100.000.000
soma = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
workers = []

worker = Process()
for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, entrada,))
    workers.append(worker)
for worker in workers :
    worker.start()
lista = []

for i in range(PROCS):
    lista.append(saida.recv()) # pega os resultados do pipe assim que pronto
print(sum(lista)*step)

for worker in workers :
    worker.join()
toc = time.time()

print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  Start: 0
End:  12499999
Start: Start:   25000000Start: 
 Start:  50000000End: 
12500000Start:  Start: End: 87500000 62500000
  
75000000
End: 37499999 
37500000End:   8749999962499999


End: 99999999End: End:  
 24999999 
74999999

49999999
30131554.06585157
41950772.29209049
37980679.66069881
49741993.88023793
33960714.81988813
48249463.66733536
45516799.350641705
26627263.510329496
3.1415924124707346
Tempo Pi: 2.66049266 s


In [34]:
# paralelo usando pipe e Lock. O lock impede que uma coisa interrompa a outra, 
# evitando erro de escrever no pipe ou outra variavel ao mesmo tempo
# (olha como o print saiu certo agora)
from multiprocessing import Process, Pipe, Lock
import sys
import time
PROCS = 8
def pi(start, end, step, entrada, lock):
    with lock:
        print ("Start: ", str(start))
        print ("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    
    with lock:
        entrada.send(soma) # manda os resultados pro pipe assim que pronto
        print(soma)
    
lock = Lock()
entrada , saida = Pipe()
num_steps = 100000000 #100.000.000
soma = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()

workers = []
worker = Process()
for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, entrada, lock, ))
    workers.append(worker)
for worker in workers :
    worker.start()
lista = []

for i in range(PROCS):
    lista.append(saida.recv()) # pega os resultados do pipe assim que pronto
print(sum(lista)*step)

for worker in workers :
    worker.join()
toc = time.time()

print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
End:  12499999
Start:  12500000
End:  24999999
Start:  25000000
End:  37499999
Start:  37500000
End:  49999999
Start:  50000000
End:  62499999
Start:  62500000
End:  74999999
Start:  87500000
End:  99999999
Start:  75000000
End:  87499999
45516799.350641705
48249463.66733536
49741993.88023793
26627263.510329496
37980679.66069881
41950772.29209049
30131554.06585157
33960714.81988813
3.1415924124707346
Tempo Pi: 2.73647952 s


In [1]:
# usando queue ao inves de pipe pras saidas. queue não tem problema se não usar lock, mais seguro.
from multiprocessing import Process, Lock, Queue
import sys
import time
PROCS = 8
def pi(start, end, step, lock, saida):
    with lock:
        print ("Start: ", str(start))
        print ("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = (soma + 4.0/(1.0+x*x))
    saida.put(soma) # manda os resultados pra queue assim que pronto
    
lock = Lock()
num_steps = 100000000 #100.000.000
soma = 0.0
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
saida = Queue()

workers = []
worker = Process()
for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, lock, saida))
    workers.append(worker)
for worker in workers :
    worker.start()
lista = []

for i in range(PROCS):
    lista.append(saida.get())
print(sum(lista)*step)

for worker in workers :
    worker.join()
toc = time.time()

print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
End:  12499999
Start:  12500000
End:  24999999
Start:  25000000
End:  37499999
Start:  37500000
End:  49999999
Start:  62500000
End:  74999999
Start:  50000000
End:  62499999
Start:  75000000
End:  87499999
Start:  87500000
End:  99999999
3.141592412470734
Tempo Pi: 2.60396457 s


In [20]:
# variavel compartilhada Value(), tem lock embutido
import time
from multiprocessing import Process, Value
def func(val):
    for i in range(50):
        time.sleep(0.01)
        with val.get_lock():
            val.value += 1



v = Value('i', 0, lock=True) # i aqui é de inteiro
procs = [Process(target=func, args=(v,)) for i in range(10)]
for p in procs: p.start()
for p in procs: p.join()
print (v.value)

500


In [36]:
# usando variavel global compartilhada Value()
from multiprocessing import Process, Value
import sys
import time
PROCS = 8
def pi(start, end, step, soma_geral):
    print ("Start: ", str(start))
    print ("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i+0.5) * step
        soma = soma + 4.0/(1.0+x*x)
    with soma_geral.get_lock():
        soma_geral.value += soma*step # manda os resultados pra queue assim que pronto
    
soma_geral = Value("d", lock=True)  # d de double
num_steps = 100000000 #100.000.000
step = 1.0/num_steps
proc_size = num_steps // PROCS
tic = time.time()
saida = Queue()

workers = []
worker = Process()
for i in range(PROCS):
    worker = Process(target=pi, args=(i*proc_size, (i+1)*proc_size - 1, step, soma_geral, ))
    workers.append(worker)
for worker in workers :
    worker.start()
lista = []


for worker in workers :
    worker.join()

print(soma_geral.value)
toc = time.time()

print ("Tempo Pi: %.8f s" %(toc-tic))

Start:  0
Start: End: Start:  Start:   12500000 Start:  2500000050000000
3750000012499999
Start: 
 End: End:  62499999End: 62500000  37499999



24999999End: 
 74999999Start: 

End:   Start:  750000008750000049999999

End: 
 End:  8749999999999999

3.1415924124707346
Tempo Pi: 2.66102242 s


In [ ]:
# usando pool

from multiprocessing import Pool
import sys
import time


def pi(start, end, step):
    print("Start: ", str(start))
    print("End: ", str(end))
    soma = 0.0
    for i in range(start, end):
        x = (i + 0.5) * step
        soma = (soma + 4.0 / (1.0 + x * x))
    return (soma)

if (__name__ == "__main__") or True:
    PROCS = 8
    num_steps = 100000000  # 100.000.000
    soma = 0.0
    step = 1.0 / num_steps
    proc_size = num_steps // PROCS
    tic = time.time()

    p = Pool(PROCS)
    lista = []

    for i in range(PROCS):
        lista.append(p.apply_async(pi, args=((i * proc_size, (i + 1) * proc_size - 1, step,))))

    output = [p.get() for p in lista]

    print(sum(output) * step)
    toc = time.time()

    print("Tempo Pi: %.8f s" % (toc - tic))